## Final Project Submission

Please fill out:
* Student name: Charlie Losche
* Student pace: full time
* Scheduled project review date/time: 8/1/2022
* Instructor name: David Elliot
* Blog post URL:N/A


In [1]:
# Your code here - remember to use markdown cells for comments as well!

In [2]:
#import 

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import seaborn as sns
from sklearn.linear_model import LinearRegression
from mpl_toolkits import mplot3d
import sklearn.metrics as metrics
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

%matplotlib inline

# Attempting Sklearn predictive model with our current regression model

In [3]:
df = pd.read_csv('./data/kc_house_data.csv',
                parse_dates=['date'])

df.drop(['id', 'floors',
         'heat_source','lat','long'], axis=1, inplace=True)

# Create zip from address
df['zip'] = [x.split(',')[2][-5:] for x in df['address']] 

#https://www.zillow.com/browse/homes/wa/king-county/  - remove all records with zipcodes that dont start with '98'
df = df[df.zip.str.startswith(('98'))]

df['city'] = [x.split(',')[1][0:] for x in df['address']]

#create sale year column from datetime date info
df['sale_year'] = pd.DatetimeIndex(df['date']).year.astype(int)

df['sale_year'].value_counts()


#create new column - age of house
df['home_age'] = (df['sale_year']+1) - df['yr_built']

#create new column - time since reno
df['yr_from_reno'] = (df['sale_year']+1) - df['yr_renovated']

# create filter for yr_from_reno to ID houses that haven't been renovated
reno_filter = df['yr_from_reno'] > 2000

# create new column for design_age to see time since build if no reno, or time since reno if renovated
# first set to reno age
df['design_age'] = df['yr_from_reno']
#then update columns where there was no reno to age of the home
df['design_age'].loc[reno_filter] = df['home_age']

#some houses were sold before they were built - we need to change those records to 0

prebuild_filter = df['design_age'] < 1

df['design_age'].loc[prebuild_filter] = 1


grade_cat = df[['grade']]
grade_categories = ['1 Cabin', '2 Substandard', '3 Poor', '4 Low', '5 Fair', '6 Low Average',
                    '7 Average', '8 Good', '9 Better', '10 Very Good', '11 Excellent', '12 Luxury', '13 Mansion']
encoder_grade = OrdinalEncoder(categories=[grade_categories])
encoder_grade.fit(grade_cat)
grade_encoded_train = encoder_grade.transform(grade_cat)
grade_encoded_train = grade_encoded_train.flatten()
df["grade"] = grade_encoded_train

C:\Users\alira\Anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\alira\Anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [4]:
len(df['city'].value_counts())

63

Updating the df to only include homes tham meet minimum code requirements - Average grade of construction and design, and up to homes where finish work is better and more design quality

In [5]:
grade_filter1 = df["grade"] > 5 
grade_filter2 = df["grade"] < 11

In [6]:
df = df.loc[grade_filter1 & grade_filter2]

In [7]:
len(df['city'].value_counts())

61

In [8]:
df['city'].value_counts()

 Seattle            8081
 Renton             1633
 Kent               1466
 Bellevue           1417
 Auburn             1288
                    ... 
 Snoqualmie Pass       1
 Dilworth              1
 Lakeland North        1
 Snohomish             1
 Cottage Lake          1
Name: city, Length: 61, dtype: int64

In [9]:
df_city = df.groupby('city').count()['price'].reset_index()
city_columns = df_city[ df_city['price'] >= 30 ].transpose()
city_columns.columns = city_columns.iloc[0]
len(city_columns.columns)

36

In [10]:
drop_city = df.groupby('city').count()['price'].reset_index()
drop_city_columns = drop_city[ drop_city['price'] < 30 ].transpose()
drop_city_columns.columns = drop_city_columns.iloc[0]
drop_city_columns.columns

Index([' Algona', ' Ames Lake', ' Arlington', ' Beaux Arts', ' Cottage Lake',
       ' Dash Point', ' Dilworth', ' Eatonville', ' Fairwood', ' Hobart',
       ' Hunts Point', ' Lakeland North', ' Marysville', ' Milton', ' Monroe',
       ' Preston', ' Puyallup', ' Skykomish', ' Snohomish', ' Snoqualmie Pass',
       ' Spanaway', ' Tulalip', ' Union Hill-Novelty Hill', ' Vancouver',
       ' Yarrow Point'],
      dtype='object', name='city')

In [11]:
drop_city_columns

city,Algona,Ames Lake,Arlington,Beaux Arts,Cottage Lake,Dash Point,Dilworth,Eatonville,Fairwood,Hobart,...,Preston,Puyallup,Skykomish,Snohomish,Snoqualmie Pass,Spanaway,Tulalip,Union Hill-Novelty Hill,Vancouver,Yarrow Point
city,Algona,Ames Lake,Arlington,Beaux Arts,Cottage Lake,Dash Point,Dilworth,Eatonville,Fairwood,Hobart,...,Preston,Puyallup,Skykomish,Snohomish,Snoqualmie Pass,Spanaway,Tulalip,Union Hill-Novelty Hill,Vancouver,Yarrow Point
price,25,1,4,10,1,1,1,2,1,4,...,4,2,6,1,1,2,4,2,2,8


In [12]:
drop_city = list(drop_city_columns.columns)

drop_city

[' Algona',
 ' Ames Lake',
 ' Arlington',
 ' Beaux Arts',
 ' Cottage Lake',
 ' Dash Point',
 ' Dilworth',
 ' Eatonville',
 ' Fairwood',
 ' Hobart',
 ' Hunts Point',
 ' Lakeland North',
 ' Marysville',
 ' Milton',
 ' Monroe',
 ' Preston',
 ' Puyallup',
 ' Skykomish',
 ' Snohomish',
 ' Snoqualmie Pass',
 ' Spanaway',
 ' Tulalip',
 ' Union Hill-Novelty Hill',
 ' Vancouver',
 ' Yarrow Point']

In [13]:
len(df)

25777

Now we remove all home sales in cities that had less than a total of 30 sales.

In [14]:
for city in drop_city:
    df = df[~df.city.str.contains(city)]

In [15]:
len(df)

25655

Updating the df to only include homes tham meet minimum code requirements - Average grade of construction and design, and up to homes where finish work is better and more design quality

In [16]:
len(df['city'].value_counts())

36

### Binning data to sqft living over 500 sqft

https://www.seattle.gov/Documents/Departments/SDCI/Codes/SingleFamilyZoningSummary.pdf

https://getjerry.com/questions/how-many-square-feet-is-a-2-car-garage

According to the zoning proposal above, small, single family (larger cottage style) homes have a total square feet of 1,400 - assuming garage is the only non-living area, and the average 2 car garage is 360 sqft (from Jerry above)

- minimim living sqft should be 1000 (giving extra flex room for garage)

Given the fact that we're not building mansions, just single family homes, if we build detached homes with total area of 2200 sqft, we'd get access to the FAR preservation incentive.

In [17]:
len(df)

25655

In [18]:
sqft_filter1 = df["sqft_living"] > 1100

In [19]:
df = df.loc[sqft_filter1]

In [20]:
len(df)

24257

In [21]:
y = df["price"]
X = df.drop("price", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=43)

In [22]:
print(f"X_train is a DataFrame with {X_train.shape[0]} rows and {X_train.shape[1]} columns")
print(f"y_train is a Series with {y_train.shape[0]} values")

print(f"X_test is a DataFrame with {X_test.shape[0]} rows and {X_test.shape[1]} columns")
print(f"y_test is a Series with {y_test.shape[0]} values")

X_train is a DataFrame with 18192 rows and 25 columns
y_train is a Series with 18192 values
X_test is a DataFrame with 6065 rows and 25 columns
y_test is a Series with 6065 values


In [25]:

# Quasi-categorical(0.5 + 0.5 bathrooms != 1 bathroom). Change 0 bathroom to 1 bathroom. Round up other values.
X_train.loc[X_train['bathrooms'] == 0, 'bathrooms'] = 1
X_train['bathrooms'] = X_train['bathrooms'].apply(np.ceil)


# Create binary substitutes for columns
X_train['basement_binary'] = [1 if sq >  0  else 0 for sq in X_train['sqft_basement']]
X_train['garage_binary'] = [1 if sq >  0  else 0 for sq in X_train['sqft_garage']]
X_train['patio_binary'] = [1 if sq >  0  else 0 for sq in X_train['sqft_patio']]
X_train['waterfront_binary'] = [1 if sq == 'YES'  else 0 for sq in X_train['waterfront']]
X_train['PublicSewer_binary'] = [1 if sq == 'PUBLIC' else 0 for sq in X_train['sewer_system']]
X_train['yrenovated_binary'] = [1 if sq >  0 else 0 for sq in X_train['yr_renovated']]


# Create binary values for greenbelt, nuisance

nuisance_cat = X_train[['nuisance']]
encoder_nuisance = OrdinalEncoder()
encoder_nuisance.fit(nuisance_cat)
nuisance_encoded_train = encoder_nuisance.transform(nuisance_cat)
nuisance_encoded_train = nuisance_encoded_train.flatten()
X_train["nuisance"] = nuisance_encoded_train

# Create Ordinal Values for condition
# Using OneHotEncoder
condition_cat = X_train[['condition']]
condition_categories = ['Poor', 'Fair', 'Average', 'Good', 'Very Good']
encoder_condition = OrdinalEncoder(categories=[condition_categories])
encoder_condition.fit(condition_cat)
condition_encoded_train = encoder_condition.transform(condition_cat)
condition_encoded_train = condition_encoded_train.flatten()
X_train["condition"] = condition_encoded_train

# Create Ordinal Values for view
# Using OneHotEncoder
view_cat = X_train[['view']]
view_categories = ['NONE', 'FAIR', 'AVERAGE', 'GOOD', 'EXCELLENT']
encoder_view = OrdinalEncoder(categories=[view_categories])
encoder_view.fit(view_cat)
view_encoded_train = encoder_view.transform(view_cat)
view_encoded_train = view_encoded_train.flatten()
X_train["view"] = view_encoded_train


# extracted from df
# (double brackets due to shape expected by OHE)
city_train = X_train[["city"]]

ohe_city = OneHotEncoder(categories="auto",
                    sparse=False,
                    handle_unknown="ignore")

# (3) Fit the encoder on fireplace_qu_train
ohe_city.fit(city_train)

# (2) Instantiate a OneHotEncoder with categories="auto",
# sparse=False, and handle_unknown="ignore"
city_encoded_train = ohe_city.transform(city_train)

# Visually inspect fireplace_qu_encoded_train
city_encoded_train

# (5a) Make the transformed data into a dataframe
city_encoded_train = pd.DataFrame(
    # Pass in NumPy array
    city_encoded_train,
    # Set the column names to the categories found by OHE
    columns=ohe_city.categories_[0],
    # Set the index to match X_test's index
    index=X_train.index
)

# (5b) Drop original FireplaceQu column
X_train.drop("city", axis=1, inplace=True)

# (5c) Concatenate the new dataframe with current X_train
X_train = pd.concat([X_train, city_encoded_train], axis=1)


# Create 'total_sqft' by combining sqft_above + sqft_basement + 
# sqft_garage + sqft_patio
X_train['total_sqft'] = X_train['sqft_above'] + X_train['sqft_basement'] + X_train['sqft_garage'] + X_train['sqft_patio']

X_train['weighted__livsqft'] = X_train['sqft_living'] * X_train['grade']

X_train['date'] = pd.to_datetime(X_train['date'], format='%Y-%m-%d')

X_train['date'].head()

#create sale year column from datetime date info
X_train['sale_year'] = pd.DatetimeIndex(X_train['date']).year.astype(int)

X_train['sale_year'].value_counts()

#Binning by decade

X_train['design_decade'] = pd.cut(x=X_train['design_age'], bins=[0, 3, 13, 23, 33, 43, 53, 63, 73,
                                                         83, 93, 103, 113, 123],
                    labels=['2020s', '2010s', '2000s', '1990s', '1980s', '1970s', '1960s',
                            '1950s', '1940s', '1930s', '1920s', '1910s', '1900s'])

X_train.design_decade.value_counts()

labels_ordered = ['2020s', '2010s', '2000s', '1990s', '1980s', '1970s', '1960s','1950s', '1940s', '1930s', '1920s', '1910s', '1900s']

labels_ordered.reverse()

labels_ordered

# Create Ordinal Values for view
# Using OneHotEncoder
age_cat = X_train[['design_decade']]
age_categories = labels_ordered
encoder_age = OrdinalEncoder(categories=[age_categories])
encoder_age.fit(age_cat)
encoder_age.categories_[0]

age_encoded_train = encoder_age.transform(age_cat)
age_encoded_train = age_encoded_train.flatten()
X_train["design_decade"] = age_encoded_train

X_train["design_decade"]

X_train['yard_size_ratio'] = (X_train['total_sqft'] / X_train['sqft_lot']) * 100

list(X_train.columns)



ValueError: could not convert string to float: 'Poor'

In [ ]:
X_train['view_grade'] = (X_train['view'] + 1) * X_train['grade']
X_train['waterfront_grade'] = (X_train['waterfront_binary'] + 1)  * X_train['grade']
X_train['design_decade_grade'] = (X_train['design_decade'] + 1)  * X_train['grade']
X_train['nuisance_grade'] = (X_train['nuisance'] + 1)  * X_train['grade']
X_train['garage_binary_grade'] = (X_train['garage_binary'] + 1)  * X_train['grade']
X_train['basement_binary_grade'] = (X_train['basement_binary'] + 1)  * X_train['grade']
X_train['patio_binary_grade'] = (X_train['patio_binary'] + 1)  * X_train['grade']
X_train['PublicSewer_binary_grade'] = (X_train['PublicSewer_binary'] + 1)  * X_train['grade']
X_train['condition_grade'] = (X_train['condition'] + 1)  * X_train['grade']

In [ ]:
relevant = [' Auburn', ' Bellevue', ' Black Diamond', ' Bothell', ' Burien',
           ' Carnation', ' Clyde Hill', ' Covington', ' Des Moines', ' Duvall',
           ' Enumclaw', ' Fall City', ' Federal Way', ' Issaquah', ' Kenmore',
           ' Kent', ' Kirkland', ' Lake Forest Park', ' Maple Valley', ' Medina',
           ' Mercer Island', ' Newcastle', ' Normandy Park', ' North Bend',
           ' Pacific', ' Ravensdale', ' Redmond', ' Renton', ' Sammamish',
           ' SeaTac', ' Seattle', ' Shoreline', ' Snoqualmie', ' Tukwila',
           ' Vashon', ' Woodinville',
            'weighted__livsqft', 'design_decade', 'yard_size_ratio', 'view_grade',
            'waterfront_grade', 'design_decade_grade', 'nuisance_grade', 'garage_binary_grade',
            'basement_binary_grade', 'patio_binary_grade', 'PublicSewer_binary_grade', 'condition_grade']

X_train1 = X_train[relevant]

X_train = X_train[relevant]

In [ ]:
X_train.view_grade.value_counts()

In [ ]:
model_train = sm.OLS(y_train, sm.add_constant(X_train)).fit()
results_train = model_train.summary()
results_train

In [ ]:
X_test.head()

In [ ]:
# Quasi-categorical(0.5 + 0.5 bathrooms != 1 bathroom). Change 0 bathroom to 1 bathroom. Round up other values.
X_test.loc[X_test['bathrooms'] == 0, 'bathrooms'] = 1
X_test['bathrooms'] = X_test['bathrooms'].apply(np.ceil)


# Create binary substitutes for columns
X_test['basement_binary'] = [1 if sq >  0  else 0 for sq in X_test['sqft_basement']]
X_test['garage_binary'] = [1 if sq >  0  else 0 for sq in X_test['sqft_garage']]
X_test['patio_binary'] = [1 if sq >  0  else 0 for sq in X_test['sqft_patio']]
X_test['waterfront_binary'] = [1 if sq == 'YES'  else 0 for sq in X_test['waterfront']]
X_test['PublicSewer_binary'] = [1 if sq == 'PUBLIC' else 0 for sq in X_test['sewer_system']]
X_test['yrenovated_binary'] = [1 if sq >  0 else 0 for sq in X_test['yr_renovated']]


# Create binary values for greenbelt, nuisance
# Using OneHotEncoder

greenbelt_cat = X_test[['greenbelt']]
greenbelt_encoded_train = encoder_greenbelt.transform(greenbelt_cat)
greenbelt_encoded_train = greenbelt_encoded_train.flatten()
# greenbelt_encoded_train
X_test["greenbelt"] = greenbelt_encoded_train

nuisance_cat = X_test[['nuisance']]
nuisance_encoded_train = encoder_nuisance.transform(nuisance_cat)
nuisance_encoded_train = nuisance_encoded_train.flatten()
X_test["nuisance"] = nuisance_encoded_train

# Create Ordinal Values for condition
# Using OneHotEncoder
condition_cat = X_test[['condition']]
condition_encoded_train = encoder_condition.transform(condition_cat)
condition_encoded_train = condition_encoded_train.flatten()
X_test["condition"] = condition_encoded_train

# Create Ordinal Values for view
# Using OneHotEncoder
view_cat = X_test[['view']]
view_encoded_train = encoder_view.transform(view_cat)
view_encoded_train = view_encoded_train.flatten()
X_test["view"] = view_encoded_train


# (1) Create a variable zip
# extracted from df
# (double brackets due to shape expected by OHE)
city_train = X_test[["city"]]

city_encoded_train = ohe_city.transform(city_train)

# Visually inspect fireplace_qu_encoded_train
city_encoded_train

# (5a) Make the transformed data into a dataframe
city_encoded_train = pd.DataFrame(
    # Pass in NumPy array
    city_encoded_train,
    # Set the column names to the categories found by OHE
    columns=ohe_city.categories_[0],
    # Set the index to match X_test's index
    index=X_test.index
)

# Run this cell without changes

# (5b) Drop original FireplaceQu column
X_test.drop("city", axis=1, inplace=True)

# Run this cell without changes

# (5c) Concatenate the new dataframe with current X_test
X_test = pd.concat([X_test, city_encoded_train], axis=1)

# Visually inspect X_test
X_test.head()

In [ ]:

# (1) Create a variable zip
# extracted from df
# (double brackets due to shape expected by OHE)
fireplace_qu_train = X_test[["zip"]]

# Replace None with appropriate code

# (4) Transform fireplace_qu_train using the encoder and
# assign the result to fireplace_qu_encoded_train
fireplace_qu_encoded_train = ohe_zip.transform(fireplace_qu_train)

# Visually inspect fireplace_qu_encoded_train
fireplace_qu_encoded_train

# Run this cell without changes

# (5a) Make the transformed data into a dataframe
fireplace_qu_encoded_train = pd.DataFrame(
    # Pass in NumPy array
    fireplace_qu_encoded_train,
    # Set the column names to the categories found by OHE
    columns=ohe_zip.categories_[0],
    # Set the index to match X_test's index
    index=X_test.index
)

# Visually inspect new dataframe
fireplace_qu_encoded_train

# Run this cell without changes

# (5b) Drop original FireplaceQu column
X_test.drop("zip", axis=1, inplace=True)

# Visually inspect X_test

# Run this cell without changes

# (5c) Concatenate the new dataframe with current X_test
X_test = pd.concat([X_test, fireplace_qu_encoded_train], axis=1)

# Visually inspect X_test
X_test.head()

# Create 'total_sqft' by combining sqft_above + sqft_basement + 
# sqft_garage + sqft_patio
X_test['total_sqft'] = X_test['sqft_above'] + X_test['sqft_basement'] + X_test['sqft_garage'] + X_test['sqft_patio']

X_test['weighted__livsqft'] = X_test['sqft_living'] * X_test['grade']

X_test['date'] = pd.to_datetime(X_test['date'], format='%Y-%m-%d')

X_test['date'].head()

#create sale year column from datetime date info
X_test['sale_year'] = pd.DatetimeIndex(X_test['date']).year.astype(int)

X_test['sale_year'].value_counts()

#Binning by decade

X_test['design_decade'] = pd.cut(x=X_test['design_age'], bins=[0, 3, 13, 23, 33, 43, 53, 63, 73,
                                                         83, 93, 103, 113, 123],
                    labels=['2020s', '2010s', '2000s', '1990s', '1980s', '1970s', '1960s',
                            '1950s', '1940s', '1930s', '1920s', '1910s', '1900s'])

X_test.design_decade.value_counts()

labels_ordered = ['2020s', '2010s', '2000s', '1990s', '1980s', '1970s', '1960s','1950s', '1940s', '1930s', '1920s', '1910s', '1900s']

labels_ordered.reverse()

labels_ordered

# Create Ordinal Values for view
# Using OneHotEncoder
age_cat = X_test[['design_decade']]
age_categories = labels_ordered
age_encoded_train = encoder_age.transform(age_cat)
age_encoded_train = age_encoded_train.flatten()
X_test["design_decade"] = age_encoded_train

X_test["design_decade"]

X_test['yard_size_ratio'] = (X_test['total_sqft'] / X_test['sqft_lot']) * 100

list(X_test.columns)

In [ ]:
X_test['view_grade'] = (X_test['view'] + 1) * X_test['grade']
X_test['waterfront_grade'] = (X_test['waterfront_binary'] + 1)  * X_test['grade']
X_test['design_decade_grade'] = (X_test['design_decade'] + 1)  * X_test['grade']
X_test['nuisance_grade'] = (X_test['nuisance'] + 1)  * X_test['grade']
X_test['garage_binary_grade'] = (X_test['garage_binary'] + 1)  * X_test['grade']
X_test['basement_binary_grade'] = (X_test['basement_binary'] + 1)  * X_test['grade']
X_test['patio_binary_grade'] = (X_test['patio_binary'] + 1)  * X_test['grade']
X_test['PublicSewer_binary_grade'] = (X_test['PublicSewer_binary'] + 1)  * X_test['grade']
X_test['condition_grade'] = (X_test['condition'] + 1)  * X_test['grade']

In [ ]:
relevant = [' Auburn', ' Bellevue', ' Black Diamond', ' Bothell', ' Burien',
           ' Carnation', ' Clyde Hill', ' Covington', ' Des Moines', ' Duvall',
           ' Enumclaw', ' Fall City', ' Federal Way', ' Issaquah', ' Kenmore',
           ' Kent', ' Kirkland', ' Lake Forest Park', ' Maple Valley', ' Medina',
           ' Mercer Island', ' Newcastle', ' Normandy Park', ' North Bend',
           ' Pacific', ' Ravensdale', ' Redmond', ' Renton', ' Sammamish',
           ' SeaTac', ' Seattle', ' Shoreline', ' Snoqualmie', ' Tukwila',
           ' Vashon', ' Woodinville',
            'weighted__livsqft', 'design_decade', 'yard_size_ratio', 'view_grade',
            'waterfront_grade', 'design_decade_grade', 'nuisance_grade', 'garage_binary_grade',
            'basement_binary_grade', 'patio_binary_grade', 'PublicSewer_binary_grade', 'condition_grade']

X_test = X_test[relevant]

In [ ]:
# relevant = ['view', 'condition', ' Auburn', ' Bellevue', ' Black Diamond', ' Bothell', ' Burien',
#            ' Carnation', ' Clyde Hill', ' Covington', ' Des Moines', ' Duvall',
#            ' Enumclaw', ' Fall City', ' Federal Way', ' Issaquah', ' Kenmore',
#            ' Kent', ' Kirkland', ' Lake Forest Park', ' Maple Valley', ' Medina',
#            ' Mercer Island', ' Newcastle', ' Normandy Park', ' North Bend',
#            ' Pacific', ' Ravensdale', ' Redmond', ' Renton', ' Sammamish',
#            ' SeaTac', ' Seattle', ' Shoreline', ' Snoqualmie', ' Tukwila',
#            ' Vashon', ' Woodinville', 'weighted__livsqft', 'design_decade', 'nuisance','garage_binary',
#             'basement_binary', 'patio_binary','PublicSewer_binary', 'waterfront_binary', 'yard_size_ratio', 'grade']

# X_test = X_test[relevant]

In [ ]:
model_test = sm.OLS(y_test, sm.add_constant(X_test)).fit()
results_test = model_test.summary()
results_test

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)
model.score(X_test, y_test)

## Error Based Metrics

### Descriptive Statistices

In [ ]:
print('the Mean price for Train is:', y_train.mean())
print('the Mean price for Test is:', y_test.mean())
print()
print('the standard deviation in price for Train is:', y_train.std())
print('the standard deviation in price for Test is:', y_test.std())

### Model Evaluation

In [ ]:
print('the adjusted R-squared value for Train is:', model_train.rsquared_adj)
print('the adjusted R-squared value for Test is:', model_test.rsquared_adj)
print()
print('the F-statistic p-value for Train is:', model_train.f_pvalue)
print('the F-statistic p-value for Test is:', model_test.f_pvalue)

**R Squared and P value Interpretation**

With an adjusted R2 for both train and test at .643 and .692 respectively, our model explains 64% to 69% of the variance in price

As for the P values of the F statistic, both our training and testing datasets have shown that our model has statistical significance, we reject the Null that our model's features have no impact on price (target variable)

### Mean Absolute Error

In [ ]:
train_mae = model_train.resid.abs().sum() / len(y_train)
test_mae = model_test.resid.abs().sum() / len(y_test)
print('the Mean Absolute Error for Train is:', train_mae)
print('the Mean Absolute Error for Test is:', test_mae)

### Root Mean Squared Error

In [ ]:
train_rmse = ((model_train.resid ** 2).sum() / len(y_train)) ** 0.5
test_rmse = ((model_test.resid ** 2).sum() / len(y_test)) ** 0.5
print('the Root Mean Squared Error for Train is:', train_rmse)
print('the Root Mean Squared Error for Test is:', test_rmse)

**MAE Interpretation**

For this specific MAE evaluation, the MEA scores mean that our model is off by about $270k dollars on a given prediction on the price.

The RMSE values of 49k and 41k indicate our model is off by $410-490k on a given prediction on price.

**Error Contextualization**

Regarding the MAE, an MAE of 270,000 is 20 percent of the average home value of 1.1 Million. 

Regarding the RMSE, an RMSE of 490,000 is 40 percent of the average home value of 1.1 Million. 

## Log transformations

In [ ]:
ylog_train = np.log(y_train)
ylog_test = np.log(y_test)

log_train_model = sm.OLS(ylog_train,sm.add_constant(X_train)).fit()
log_test_model = sm.OLS(ylog_test,sm.add_constant(X_test)).fit()

In [ ]:
log_train_model.summary()

In [ ]:
log_test_model.summary()

In [ ]:
model_train.params['basement_binary_grade']

In [ ]:
def log_explain(column_list, model):
    column_list.insert(0, 'const')
    
    for e in column_list:
        log_val = model.params[e]
        if log_val > 0:
            print(f"""
            A one-unit increase in the {e} variable corresponds
            to an increase in price by a factor of {round(np.exp(log_val), 3)},
            or {round(np.exp(log_val) - 1, 3)}%.
            """) 
        else:
            print(f"""
            A one-unit increase in the {e} variable corresponds
            to a decrease price to {round(np.exp(log_val), 3)}% of its prior value,
            or a decrease of {round((log_val * 100), 3)}%.
            """) 
            

In [ ]:
X_train_col = X_train.columns

X_train_col

In [ ]:
log_explain(X_train_col, log_train_model)

# Visualizations

No filters

In [ ]:
df = pd.read_csv('./data/kc_house_data.csv',
                parse_dates=['date'])

df.drop(['id', 'floors',
         'heat_source','lat','long'], axis=1, inplace=True)

# Create zip from address
df['zip'] = [x.split(',')[2][-5:] for x in df['address']] 

#https://www.zillow.com/browse/homes/wa/king-county/  - remove all records with zipcodes that dont start with '98'

df['city'] = [x.split(',')[1][0:] for x in df['address']]

#create sale year column from datetime date info
df['sale_year'] = pd.DatetimeIndex(df['date']).year.astype(int)

df['sale_year'].value_counts()


#create new column - age of house
df['home_age'] = (df['sale_year']+1) - df['yr_built']

#create new column - time since reno
df['yr_from_reno'] = (df['sale_year']+1) - df['yr_renovated']

# create filter for yr_from_reno to ID houses that haven't been renovated
reno_filter = df['yr_from_reno'] > 2000

# create new column for design_age to see time since build if no reno, or time since reno if renovated
# first set to reno age
df['design_age'] = df['yr_from_reno']
#then update columns where there was no reno to age of the home
df['design_age'].loc[reno_filter] = df['home_age']

#some houses were sold before they were built - we need to change those records to 0

prebuild_filter = df['design_age'] < 1

df['design_age'].loc[prebuild_filter] = 1


grade_cat = df[['grade']]
grade_categories = ['1 Cabin', '2 Substandard', '3 Poor', '4 Low', '5 Fair', '6 Low Average',
                    '7 Average', '8 Good', '9 Better', '10 Very Good', '11 Excellent', '12 Luxury', '13 Mansion']
encoder_grade = OrdinalEncoder(categories=[grade_categories])
encoder_grade.fit(grade_cat)
grade_encoded_train = encoder_grade.transform(grade_cat)
grade_encoded_train = grade_encoded_train.flatten()
df["grade"] = grade_encoded_train

X_train = df

# Quasi-categorical(0.5 + 0.5 bathrooms != 1 bathroom). Change 0 bathroom to 1 bathroom. Round up other values.
X_train.loc[X_train['bathrooms'] == 0, 'bathrooms'] = 1
X_train['bathrooms'] = X_train['bathrooms'].apply(np.ceil)


# Create binary substitutes for columns
X_train['basement_binary'] = [1 if sq >  0  else 0 for sq in X_train['sqft_basement']]
X_train['garage_binary'] = [1 if sq >  0  else 0 for sq in X_train['sqft_garage']]
X_train['patio_binary'] = [1 if sq >  0  else 0 for sq in X_train['sqft_patio']]
X_train['waterfront_binary'] = [1 if sq == 'YES'  else 0 for sq in X_train['waterfront']]
X_train['PublicSewer_binary'] = [1 if sq == 'PUBLIC' else 0 for sq in X_train['sewer_system']]
X_train['yrenovated_binary'] = [1 if sq >  0 else 0 for sq in X_train['yr_renovated']]


# Create binary values for greenbelt, nuisance
# Using OneHotEncoder

greenbelt_cat = X_train[['greenbelt']]
encoder_greenbelt = OrdinalEncoder(categories=[['NO', 'YES']])
encoder_greenbelt.fit(greenbelt_cat)
greenbelt_encoded_train = encoder_greenbelt.transform(greenbelt_cat)
greenbelt_encoded_train = greenbelt_encoded_train.flatten()
# greenbelt_encoded_train
X_train["greenbelt"] = greenbelt_encoded_train

nuisance_cat = X_train[['nuisance']]
encoder_nuisance = OrdinalEncoder()
encoder_nuisance.fit(nuisance_cat)
nuisance_encoded_train = encoder_nuisance.transform(nuisance_cat)
nuisance_encoded_train = nuisance_encoded_train.flatten()
X_train["nuisance"] = nuisance_encoded_train

# Create Ordinal Values for condition
# Using OneHotEncoder
condition_cat = X_train[['condition']]
condition_categories = ['Poor', 'Fair', 'Average', 'Good', 'Very Good']
encoder_condition = OrdinalEncoder(categories=[condition_categories])
encoder_condition.fit(condition_cat)
condition_encoded_train = encoder_condition.transform(condition_cat)
condition_encoded_train = condition_encoded_train.flatten()
X_train["condition"] = condition_encoded_train

# Create Ordinal Values for view
# Using OneHotEncoder
view_cat = X_train[['view']]
view_categories = ['NONE', 'FAIR', 'AVERAGE', 'GOOD', 'EXCELLENT']
encoder_view = OrdinalEncoder(categories=[view_categories])
encoder_view.fit(view_cat)
view_encoded_train = encoder_view.transform(view_cat)
view_encoded_train = view_encoded_train.flatten()
X_train["view"] = view_encoded_train


# Create 'total_sqft' by combining sqft_above + sqft_basement + 
# sqft_garage + sqft_patio
X_train['total_sqft'] = X_train['sqft_above'] + X_train['sqft_basement'] + X_train['sqft_garage'] + X_train['sqft_patio']

X_train['weighted__livsqft'] = X_train['sqft_living'] * X_train['grade']

X_train['date'] = pd.to_datetime(X_train['date'], format='%Y-%m-%d')

X_train['date'].head()

#create sale year column from datetime date info
X_train['sale_year'] = pd.DatetimeIndex(X_train['date']).year.astype(int)

X_train['sale_year'].value_counts()

#Binning by decade

X_train['design_decade'] = pd.cut(x=X_train['design_age'], bins=[0, 3, 13, 23, 33, 43, 53, 63, 73,
                                                         83, 93, 103, 113, 123],
                    labels=['2020s', '2010s', '2000s', '1990s', '1980s', '1970s', '1960s',
                            '1950s', '1940s', '1930s', '1920s', '1910s', '1900s'])

X_train.design_decade.value_counts()

labels_ordered = ['2020s', '2010s', '2000s', '1990s', '1980s', '1970s', '1960s','1950s', '1940s', '1930s', '1920s', '1910s', '1900s']

labels_ordered.reverse()

labels_ordered

# Create Ordinal Values for view
# Using OneHotEncoder
age_cat = X_train[['design_decade']]
age_categories = labels_ordered
encoder_age = OrdinalEncoder(categories=[age_categories])
encoder_age.fit(age_cat)
encoder_age.categories_[0]

age_encoded_train = encoder_age.transform(age_cat)
age_encoded_train = age_encoded_train.flatten()
X_train["design_decade"] = age_encoded_train

X_train["design_decade"]

X_train['yard_size_ratio'] = (X_train['total_sqft'] / X_train['sqft_lot']) * 100

X_train['view_grade'] = (X_train['view'] + 1) * X_train['grade']
X_train['waterfront_grade'] = (X_train['waterfront_binary'] + 1)  * X_train['grade']
X_train['design_decade_grade'] = (X_train['design_decade'] + 1)  * X_train['grade']
X_train['nuisance_grade'] = (X_train['nuisance'] + 1)  * X_train['grade']
X_train['garage_binary_grade'] = (X_train['garage_binary'] + 1)  * X_train['grade']
X_train['basement_binary_grade'] = (X_train['basement_binary'] + 1)  * X_train['grade']
X_train['patio_binary_grade'] = (X_train['patio_binary'] + 1)  * X_train['grade']
X_train['PublicSewer_binary_grade'] = (X_train['PublicSewer_binary'] + 1)  * X_train['grade']
X_train['condition_grade'] = (X_train['condition'] + 1)  * X_train['grade']


In [ ]:
list(X_train.columns)

Basic spatial Columns

In [ ]:

relevant = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement',
            'sqft_garage', 'sqft_patio']

basic_space = X_train[relevant]

In [ ]:
basic_space_corr = basic_space.corr()

basic_space_corr

In [ ]:
df_corr=X_train.corr().abs().stack().reset_index().sort_values(0, ascending=False)

df_corr

# zip the variable name columns (Which were only named level_0 and level_1 by default) in a new column named "pairs"
df_corr['pairs'] = list(zip(df_corr.level_0, df_corr.level_1))

# set index to pairs
df_corr.set_index(['pairs'], inplace = True)

#d rop level columns
df_corr.drop(columns=['level_1', 'level_0'], inplace = True)

# rename correlation column as cc rather than 0
df_corr.columns = ['cc']

# drop duplicates. This could be dangerous if you have variables perfectly correlated with variables other than themselves.
# for the sake of exercise, kept it in.
df_corr.drop_duplicates(inplace=True)

df_corr[(df_corr.cc>.75) & (df_corr.cc <1)]

In [ ]:

plt.figure(figsize=(12, 12))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(basic_space_corr, dtype=bool))
heatmap = sns.heatmap(basic_space_corr, mask=mask, vmin=-1, vmax=1, annot=True)
heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':18}, pad=16)
heatmap.set_facecolor('white');

In [ ]:
corr = round(basic_space_corr,3)
sns.set(rc = {'figure.figsize':(40,15)})
mask = np.triu(np.ones_like(corr, dtype=bool))
res = sns.heatmap(corr, mask=mask, vmin=-1, vmax=1, annot=True)
res.set_xticklabels(res.get_xmajorticklabels(), fontsize = 15)
res.set_yticklabels(res.get_ymajorticklabels(), fontsize = 15);

In [ ]:
corr = round(basic_space_corr,3)
sns.set(rc = {'figure.figsize':(40,15)})
sns.heatmap(corr, cmap="Reds", annot=True)


Engineered spatial Columns

In [ ]:

relevant = ['price', 'total_sqft', 'weighted__livsqft', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement',
            'sqft_garage', 'sqft_patio']

eng_space = X_train[relevant]

In [ ]:
eng_space_corr = eng_space.corr()

eng_space_corr

In [ ]:
corr = round(eng_space_corr,3)
sns.set(rc = {'figure.figsize':(50,18)})
mask = np.triu(np.ones_like(corr, dtype=bool))
res = sns.heatmap(corr, mask=mask, vmin=-1, vmax=1, annot=True)
res.set_xticklabels(res.get_xmajorticklabels(), fontsize = 12)
res.set_yticklabels(res.get_ymajorticklabels(), fontsize = 12);

Basic categorical Columns

In [ ]:
list(X_train.columns)

In [ ]:
relevant = ['price', 'waterfront', 'greenbelt', 'nuisance', 'view', 'condition', 'grade','basement_binary', 'garage_binary',
            'patio_binary', 'waterfront_binary', 'PublicSewer_binary', 'yrenovated_binary','design_decade']

bas_cat = X_train[relevant]

In [ ]:
bas_cat_corr = bas_cat.corr()

bas_cat_corr

In [ ]:
corr = round(bas_cat_corr,3)
sns.set(rc = {'figure.figsize':(50,18)})
mask = np.triu(np.ones_like(corr, dtype=bool))
res = sns.heatmap(corr, mask=mask, vmin=-1, vmax=1, annot=True)
res.set_xticklabels(res.get_xmajorticklabels(), fontsize = 12)
res.set_yticklabels(res.get_ymajorticklabels(), fontsize = 12);

engineered categorical Columns

In [ ]:
list(X_train.columns)

In [ ]:
relevant = ['price', 'view_grade', 'waterfront_grade', 'design_decade_grade',
            'nuisance_grade', 'garage_binary_grade', 'basement_binary_grade',
            'patio_binary_grade', 'PublicSewer_binary_grade', 'condition_grade']

eng_cat = X_train[relevant]

In [ ]:
eng_cat_corr = eng_cat.corr()

eng_cat_corr

In [ ]:
corr = round(eng_cat_corr,3)
sns.set(rc = {'figure.figsize':(50,18)})
mask = np.triu(np.ones_like(corr, dtype=bool))
res = sns.heatmap(corr, mask=mask, vmin=-1, vmax=1, annot=True)
res.set_xticklabels(res.get_xmajorticklabels(), fontsize = 12)
res.set_yticklabels(res.get_ymajorticklabels(), fontsize = 12);

### Messing Around - weights of the variables

In [ ]:
relevant = [' Auburn', ' Bellevue', ' Black Diamond', ' Bothell', ' Burien',
           ' Carnation', ' Clyde Hill', ' Covington', ' Des Moines', ' Duvall',
           ' Enumclaw', ' Fall City', ' Federal Way', ' Issaquah', ' Kenmore',
           ' Kent', ' Kirkland', ' Lake Forest Park', ' Maple Valley', ' Medina',
           ' Mercer Island', ' Newcastle', ' Normandy Park', ' North Bend',
           ' Pacific', ' Ravensdale', ' Redmond', ' Renton', ' Sammamish',
           ' SeaTac', ' Seattle', ' Shoreline', ' Snoqualmie', ' Tukwila',
           ' Vashon', ' Woodinville',
            'weighted__livsqft', 'design_decade', 'yard_size_ratio', 'view_grade',
            'waterfront_grade', 'design_decade_grade', 'nuisance_grade', 'garage_binary_grade',
            'basement_binary_grade', 'patio_binary_grade', 'PublicSewer_binary_grade', 'condition_grade']

X_train1

In [ ]:
X_standard = X_train.apply(lambda x: (x-x.mean())/ x.std())

In [ ]:
results_standardized = sm.OLS(y, sm.add_constant(X_standard)).fit()

In [ ]:
weights = results_standardized.params.abs.sort_values()
weights

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(y = weights[1::].index, x = weights[1::].values)
plt.title("Weights for factors correlating with sale price")
plt.ylabel('Weight')
plt.show()